In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = "0"
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import pandas as pd
from tqdm.auto import tqdm
import torch
from torch import nn
import gin
gin.enter_interactive_mode()

In [2]:
use_cuda = True

In [173]:
C = FCCombinedModel(hidden_sizes=[60, 60, 60], input_dim=24, n_models=25, output_dim=1)
if use_cuda:
    C = C.cuda()

In [174]:
M = AllModels()

In [175]:
%timeit C(data.view(-1, 24, 1).expand(-1, -1, n_models))

3.79 ms ± 556 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [183]:
outC = C(data.view(-1, 24, 1).expand(-1, -1, n_models))

In [184]:
outC.shape

torch.Size([1000, 25])

In [185]:
outM = M(data)

In [186]:
assert outC.shape == outM.shape

In [187]:
(outC - outM).abs().max()

tensor(0., device='cuda:0', grad_fn=<MaxBackward1>)

In [181]:
for n, p in M.named_parameters():
    model_n = int(n[1:3])
    layer_id = int(n.split('.')[1]) // 2
    is_bias = 'bias' in n
    print(n, model_n, is_bias, layer_id)
    target_param = C.fc[layer_id]
    if is_bias:
        target_param.bias.data[:, model_n] = p
    else:
        target_param.weight.data[:, :, model_n] = p

m00.0.weight 0 False 0
m00.0.bias 0 True 0
m00.2.weight 0 False 1
m00.2.bias 0 True 1
m00.4.weight 0 False 2
m00.4.bias 0 True 2
m00.6.weight 0 False 3
m00.6.bias 0 True 3
m01.0.weight 1 False 0
m01.0.bias 1 True 0
m01.2.weight 1 False 1
m01.2.bias 1 True 1
m01.4.weight 1 False 2
m01.4.bias 1 True 2
m01.6.weight 1 False 3
m01.6.bias 1 True 3
m02.0.weight 2 False 0
m02.0.bias 2 True 0
m02.2.weight 2 False 1
m02.2.bias 2 True 1
m02.4.weight 2 False 2
m02.4.bias 2 True 2
m02.6.weight 2 False 3
m02.6.bias 2 True 3
m03.0.weight 3 False 0
m03.0.bias 3 True 0
m03.2.weight 3 False 1
m03.2.bias 3 True 1
m03.4.weight 3 False 2
m03.4.bias 3 True 2
m03.6.weight 3 False 3
m03.6.bias 3 True 3
m04.0.weight 4 False 0
m04.0.bias 4 True 0
m04.2.weight 4 False 1
m04.2.bias 4 True 1
m04.4.weight 4 False 2
m04.4.bias 4 True 2
m04.6.weight 4 False 3
m04.6.bias 4 True 3
m05.0.weight 5 False 0
m05.0.bias 5 True 0
m05.2.weight 5 False 1
m05.2.bias 5 True 1
m05.4.weight 5 False 2
m05.4.bias 5 True 2
m05.6.weigh

In [7]:
%timeit M(data, do_parallel=True)

50.5 ms ± 10.7 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [8]:
%timeit M(data)

55.5 ms ± 8.4 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [9]:
%timeit M.model_list[0](data)

2.56 ms ± 195 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [10]:
torch.get_num_threads()

5

In [11]:
torch.set_num_threads(10)